# Seismic Ray Tracing with ObsPy

This notebook demonstrates how to compute travel times and visualize ray paths using ObsPy's TauP module. We'll cover:

1. **Basic travel time calculations**
2. **Geographic coordinate conversions**
3. **Circular Earth ray path plotting**
4. **Model comparisons**
5. **Pierce point calculations**
6. **Advanced visualizations**

## What is Ray Tracing?

Ray tracing in seismology calculates how seismic waves travel through the Earth's interior. It uses 1D Earth models (spherically symmetric) to compute:
- Travel times between source and receiver
- Ray paths through Earth's layers
- Arrival angles and ray parameters

## 1. Import Required Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from obspy.taup import TauPyModel
from obspy.geodetics import gps2dist_azimuth, locations2degrees

# Set matplotlib parameters for better plots
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

print("Libraries imported successfully!")
print("Available Earth models: iasp91, prem, ak135")

## 2. Basic Travel Time Calculations

Let's start with the simplest case: calculating travel times for a given source depth and epicentral distance.

In [ ]:
def basic_travel_times(source_depth=50.0, distance_deg=30.0, phases=["P", "S"]):
    """
    Calculate basic travel times for given parameters.
    """
    # Load the IASP91 Earth model
    model = TauPyModel(model="iasp91")

    # Calculate travel times
    arrivals = model.get_travel_times(
        source_depth_in_km=source_depth,
        distance_in_degree=distance_deg,
        phase_list=phases
    )

    print(f"Source depth: {source_depth} km")
    print(f"Distance: {distance_deg}°")
    print("\nPhase arrivals:")
    print("Phase    Time (s)   Ray Parameter (s/deg)")
    print("-" * 40)

    for arrival in arrivals:
        print(f"{arrival.name:<8} {arrival.time:>7.2f}    {arrival.ray_param:>10.4f}")

    return arrivals

# Example: Basic P and S wave calculation
arrivals = basic_travel_times(source_depth=50.0, distance_deg=30.0, phases=["P", "S"])

## 3. Geographic Coordinate Calculations

In real applications, we usually have latitude and longitude coordinates for earthquakes and seismic stations.

In [ ]:
def geographic_travel_times(eq_lat, eq_lon, eq_depth, sta_lat, sta_lon):
    """
    Calculate travel times using geographic coordinates.
    """
    # Calculate distance and azimuth
    distance_m, azimuth, back_azimuth = gps2dist_azimuth(eq_lat, eq_lon, sta_lat, sta_lon)
    distance_km = distance_m / 1000.0
    distance_deg = locations2degrees(eq_lat, eq_lon, sta_lat, sta_lon)

    # Load model and calculate travel times
    model = TauPyModel(model="iasp91")
    arrivals = model.get_travel_times(
        source_depth_in_km=eq_depth,
        distance_in_degree=distance_deg,
        phase_list=["P", "S", "PP", "SS"]
    )

    print(f"Earthquake: {eq_lat}°N, {eq_lon}°E, {eq_depth} km depth")
    print(f"Station: {sta_lat}°N, {sta_lon}°E")
    print(f"Distance: {distance_km:.1f} km ({distance_deg:.2f}°)")
    print(f"Azimuth: {azimuth:.1f}°")
    print("\nTravel times:")

    for arrival in arrivals[:4]:  # Show first 4 arrivals
        print(f"{arrival.name}: {arrival.time:.1f} seconds")

    return arrivals, distance_deg

# Example: Earthquake in Japan recorded in California
arrivals, distance = geographic_travel_times(
    eq_lat=35.0, eq_lon=140.0, eq_depth=50.0,  # Japan earthquake
    sta_lat=37.0, sta_lon=-122.0               # California station
)

## 4. Circular Earth Ray Path Visualization

Now let's create beautiful circular Earth cross-sections showing ray paths through the planet's interior.

In [ ]:
def plot_circular_earth_rays(source_depth=100.0, distance_deg=60.0, phases=["P", "S"]):
    """
    Plot ray paths on a circular Earth cross-section.
    """
    model = TauPyModel(model="iasp91")

    # Get ray paths
    ray_paths = model.get_ray_paths(
        source_depth_in_km=source_depth,
        distance_in_degree=distance_deg,
        phase_list=phases
    )

    # Create figure
    fig, ax = plt.subplots(figsize=(12, 10))

    # Earth parameters
    earth_radius = 6371.0  # km
    cmb_radius = 3480.0    # Core-mantle boundary
    ic_radius = 1220.0     # Inner core boundary

    # Create angular array for Earth boundaries
    theta = np.linspace(0, np.pi, 180)  # Semicircle
    theta_full = np.linspace(0, 2*np.pi, 360)  # Full circle for filling

    # Fill Earth layers with proper colors
    x_surf_full = earth_radius * np.cos(theta_full)
    y_surf_full = earth_radius * np.sin(theta_full)
    ax.fill(x_surf_full, y_surf_full, color='saddlebrown', alpha=0.4, label='Mantle')

    x_cmb_full = cmb_radius * np.cos(theta_full)
    y_cmb_full = cmb_radius * np.sin(theta_full)
    ax.fill(x_cmb_full, y_cmb_full, color='red', alpha=0.5, label='Outer Core')

    x_ic_full = ic_radius * np.cos(theta_full)
    y_ic_full = ic_radius * np.sin(theta_full)
    ax.fill(x_ic_full, y_ic_full, color='gold', alpha=0.6, label='Inner Core')

    # Plot Earth boundaries
    x_surface = earth_radius * np.cos(theta)
    y_surface = earth_radius * np.sin(theta)
    ax.plot(x_surface, y_surface, 'k-', linewidth=3, label='Surface')

    x_cmb = cmb_radius * np.cos(theta)
    y_cmb = cmb_radius * np.sin(theta)
    ax.plot(x_cmb, y_cmb, 'r--', linewidth=2, alpha=0.8, label='Core-Mantle Boundary')

    x_ic = ic_radius * np.cos(theta)
    y_ic = ic_radius * np.sin(theta)
    ax.plot(x_ic, y_ic, 'orange', linestyle='--', linewidth=2, alpha=0.8, label='Inner Core Boundary')

    # Plot ray paths
    colors = ['blue', 'red', 'green', 'purple', 'brown', 'pink']

    for i, ray_path in enumerate(ray_paths):
        path = ray_path.path
        distances_rad = path['dist']
        depths = path['depth']
        radius = earth_radius - depths

        # Convert to Cartesian coordinates
        x_ray = radius * np.cos(distances_rad)
        y_ray = radius * np.sin(distances_rad)

        color = colors[i % len(colors)]
        ax.plot(x_ray, y_ray, color=color, linewidth=3,
                label=f"{ray_path.name} ({ray_path.time:.1f}s)")

    # Mark source and receiver
    source_radius = earth_radius - source_depth
    ax.plot(source_radius, 0, 'r*', markersize=20, markeredgecolor='black',
            markeredgewidth=1, label='Source')

    receiver_angle = distance_deg * np.pi / 180.0
    receiver_x = earth_radius * np.cos(receiver_angle)
    receiver_y = earth_radius * np.sin(receiver_angle)
    ax.plot(receiver_x, receiver_y, 'b^', markersize=15, markeredgecolor='black',
            markeredgewidth=1, label='Receiver')

    # Add distance arc
    arc_angles = np.linspace(0, receiver_angle, 50)
    arc_x = earth_radius * np.cos(arc_angles)
    arc_y = earth_radius * np.sin(arc_angles)
    ax.plot(arc_x, arc_y, 'k-', linewidth=3, alpha=0.7)

    # Formatting
    ax.set_xlim(-7200, 7200)
    ax.set_ylim(-1000, 7200)
    ax.set_aspect('equal')
    ax.set_xlabel('Distance (km)', fontsize=12)
    ax.set_ylabel('Height (km)', fontsize=12)
    ax.set_title(f'Seismic Ray Paths Through Earth\n'
                f'Source: {source_depth} km depth, Distance: {distance_deg}°', fontsize=14)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    return ray_paths

# Create ray path plot
ray_paths = plot_circular_earth_rays(source_depth=100.0, distance_deg=60.0,
                                   phases=["P", "S", "PP", "SS"])

## 5. Compare Different Earth Models

Different Earth models can give slightly different travel times. Let's compare the standard models.

In [ ]:
def compare_earth_models(source_depth=100.0, distance_deg=60.0, phase="P"):
    """
    Compare travel times between different Earth models.
    """
    models = ["iasp91", "prem", "ak135"]
    results = {}

    print(f"Comparing {phase}-wave travel times:")
    print(f"Source depth: {source_depth} km, Distance: {distance_deg}°")
    print("\nModel     Time (s)   Difference (s)")
    print("-" * 35)

    reference_time = None

    for model_name in models:
        try:
            model = TauPyModel(model=model_name)
            arrivals = model.get_travel_times(
                source_depth_in_km=source_depth,
                distance_in_degree=distance_deg,
                phase_list=[phase]
            )

            if arrivals:
                time = arrivals[0].time
                results[model_name] = time

                if reference_time is None:
                    reference_time = time
                    diff_str = "(reference)"
                else:
                    diff = time - reference_time
                    diff_str = f"{diff:+.3f}"

                print(f"{model_name:<8} {time:>7.2f}    {diff_str}")
            else:
                print(f"{model_name:<8} {'No arrival':>7}")
        except Exception as e:
            print(f"{model_name:<8} {'Error':>7}")

    return results

# Compare models
comparison = compare_earth_models(source_depth=100.0, distance_deg=60.0, phase="P")

## 6. Travel Time Curves

Travel time curves show how arrival times vary with distance. These are fundamental in seismology.

In [ ]:
def plot_travel_time_curves(source_depth=100.0, max_distance=180.0):
    """
    Create travel time curves for different phases.
    """
    model = TauPyModel(model="iasp91")
    distances = np.arange(5, max_distance, 5)
    phases = ["P", "S", "PP", "SS"]

    plt.figure(figsize=(12, 8))

    for phase in phases:
        times = []
        valid_distances = []

        for dist in distances:
            try:
                arrivals = model.get_travel_times(
                    source_depth_in_km=source_depth,
                    distance_in_degree=dist,
                    phase_list=[phase]
                )
                if arrivals:
                    times.append(arrivals[0].time)
                    valid_distances.append(dist)
            except:
                continue

        if times:
            plt.plot(valid_distances, times, 'o-', label=phase, markersize=4, linewidth=2)

    plt.xlabel('Distance (degrees)', fontsize=12)
    plt.ylabel('Travel Time (seconds)', fontsize=12)
    plt.title(f'Travel Time Curves (Source depth: {source_depth} km)', fontsize=14)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Create travel time curves
plot_travel_time_curves(source_depth=100.0, max_distance=120.0)

## 7. Pierce Point Calculations

Pierce points show where rays cross specific depth levels. This is crucial for tomography studies.

In [ ]:
def calculate_pierce_points(source_depth=10.0, distance_deg=50.0, pierce_depth=400.0):
    """
    Calculate where rays pierce a specific depth level.
    """
    model = TauPyModel(model="iasp91")

    ray_paths = model.get_ray_paths(
        source_depth_in_km=source_depth,
        distance_in_degree=distance_deg,
        phase_list=["P", "S"]
    )

    print(f"Pierce points at {pierce_depth} km depth:")
    print(f"Source: {source_depth} km, Distance: {distance_deg}°")
    print("\nPhase  Pierce Distance (deg)  Pierce Time (s)")
    print("-" * 45)

    for ray_path in ray_paths:
        path = ray_path.path
        depths = path['depth']
        distances = path['dist'] * 180.0 / np.pi  # Convert to degrees
        times = path['time']

        # Find pierce points
        pierce_indices = []
        for i in range(len(depths) - 1):
            if ((depths[i] <= pierce_depth <= depths[i+1]) or
                (depths[i+1] <= pierce_depth <= depths[i])):
                pierce_indices.append(i)

        if pierce_indices:
            # Take the first pierce point (downgoing)
            idx = pierce_indices[0]
            # Linear interpolation for accuracy
            f = (pierce_depth - depths[idx]) / (depths[idx+1] - depths[idx])
            pierce_dist = distances[idx] + f * (distances[idx+1] - distances[idx])
            pierce_time = times[idx] + f * (times[idx+1] - times[idx])

            print(f"{ray_path.name:<6} {pierce_dist:>12.2f}          {pierce_time:>8.2f}")
        else:
            print(f"{ray_path.name:<6} {'No pierce':>12}")

# Calculate pierce points
calculate_pierce_points(source_depth=10.0, distance_deg=50.0, pierce_depth=400.0)

## 8. Interactive Example: Customize Your Own Ray Paths

Try changing the parameters below to explore different scenarios!

In [ ]:
# Customize these parameters and run the cell!
YOUR_SOURCE_DEPTH = 200.0      # Try values between 0-700 km
YOUR_DISTANCE = 90.0           # Try values between 10-180 degrees
YOUR_PHASES = ["P", "PKP", "PKIKP"]  # Try different phases

print(f"Creating custom ray plot for:")
print(f"Source depth: {YOUR_SOURCE_DEPTH} km")
print(f"Distance: {YOUR_DISTANCE}°")
print(f"Phases: {YOUR_PHASES}")
print("\n" + "="*50)

# Calculate travel times
arrivals = basic_travel_times(YOUR_SOURCE_DEPTH, YOUR_DISTANCE, YOUR_PHASES)

print("\n" + "="*50)
print("Creating ray path visualization...")

# Create ray path plot
ray_paths = plot_circular_earth_rays(YOUR_SOURCE_DEPTH, YOUR_DISTANCE, YOUR_PHASES)

## 9. Summary and Next Steps

### What we've learned:

1. **Basic Travel Times**: How to calculate arrival times for different seismic phases
2. **Geographic Coordinates**: Converting real earthquake and station locations to distances
3. **Ray Path Visualization**: Creating beautiful circular Earth cross-sections
4. **Model Comparisons**: Understanding differences between Earth models
5. **Travel Time Curves**: Fundamental seismological diagrams
6. **Pierce Points**: Important for seismic tomography

### Common Seismic Phases:

- **P**: Direct P-wave through mantle
- **S**: Direct S-wave through mantle
- **PP, SS**: Waves reflected once at surface
- **PcP, ScS**: Waves reflected at core-mantle boundary
- **PKP, SKS**: Waves traveling through outer core
- **PKIKP**: Waves traveling through inner core

### Earth Models:

- **IASP91**: International Association of Seismology and Physics of the Earth's Interior 1991
- **PREM**: Preliminary Reference Earth Model
- **AK135**: Kennett & Engdahl 1995 model

### Applications:

- **Earthquake location**: Using travel time differences
- **Seismic tomography**: Mapping Earth's interior structure
- **Receiver function analysis**: Understanding crustal structure
- **Early warning systems**: Predicting wave arrivals

## 10. Exercise: Try It Yourself!

**Challenge**: Calculate travel times for a recent earthquake to stations around the world.

1. Pick a recent large earthquake (magnitude > 6.0)
2. Find its location and depth
3. Choose 3-5 seismic stations at different distances
4. Calculate P and S wave arrival times
5. Create a ray path plot showing the wave propagation

Use the functions we've created above to complete this exercise!

In [ ]:
# Your exercise solution here!

# Example earthquake (modify with real data):
earthquake = {
    'lat': 35.0,
    'lon': 140.0,
    'depth': 50.0,
    'name': 'Example Earthquake'
}

stations = [
    {'name': 'Station A', 'lat': 40.0, 'lon': 145.0},
    {'name': 'Station B', 'lat': 30.0, 'lon': 130.0},
    {'name': 'Station C', 'lat': 45.0, 'lon': 135.0}
]

print(f"Analyzing {earthquake['name']}")
print(f"Location: {earthquake['lat']}°N, {earthquake['lon']}°E")
print(f"Depth: {earthquake['depth']} km\n")

for station in stations:
    print(f"\n--- {station['name']} ---")
    arrivals, distance = geographic_travel_times(
        earthquake['lat'], earthquake['lon'], earthquake['depth'],
        station['lat'], station['lon']
    )